To run live feeding

In [ ]:
import cv2
from ultralytics import YOLO
import math


model = YOLO(r"E:\insiso_assignment\runs\detect\concrete_mix_545\weights\best.pt")


classNames = ["0", "CEMENT", "CEMENT-BAG", "GRAVEL", "SAND", "UNKNOWN"]


classColors = {
    0: (128, 128, 128),  
    1: (255, 0, 0),      
    2: (0, 165, 255),    
    3: (0, 255, 255),    
    4: (0, 255, 0),      
    5: (0, 0, 255)       
}

default_color = (255, 255, 255) 


cap = cv2.VideoCapture(0)
cap.set(3, 640) 
cap.set(4, 480) 

while True:
    success, img = cap.read()
    if not success:
        break

    results = model(img, stream=True)

    for r in results:
        boxes = r.boxes
        for box in boxes:
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

            
            cls = int(box.cls[0])
            
           
            color = classColors.get(cls, default_color)

            
            cv2.rectangle(img, (x1, y1), (x2, y2), color, 3)

            conf = math.ceil((box.conf[0] * 100)) / 100
            
           
            if cls < len(classNames):
                current_class = classNames[cls]
            else:
                current_class = f"Class {cls}"
                
            label = f'{current_class} {conf}'

            t_size = cv2.getTextSize(label, 0, fontScale=1, thickness=2)[0]
            c2 = x1 + t_size[0], y1 - t_size[1] - 3
            
            
            cv2.rectangle(img, (x1, y1), c2, color, -1, cv2.LINE_AA)  
            
           
            cv2.putText(img, label, (x1, y1 - 2), 0, 1, [255, 255, 255], thickness=1, lineType=cv2.LINE_AA)

    cv2.imshow('Concrete Mix Classifier', img)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()